In [2]:
"""
mic_hotkeys_cominit_fixed.py
Global hotkeys to control microphone volume (Windows).
This version ensures COM is initialized inside each hotkey handler thread.

Hotkeys:
  Ctrl+Alt+Up    -> increase mic volume by 5%
  Ctrl+Alt+Down  -> decrease mic volume by 5%
  Ctrl+Alt+M     -> toggle mute/unmute
  Ctrl+Alt+Q     -> stop hotkeys (clean shutdown)
"""
import sys
import time
import platform
import threading
from ctypes import POINTER, cast
from comtypes import CLSCTX_ALL, CoInitialize, CoUninitialize
from comtypes.client import CreateObject
from comtypes import GUID
from functools import wraps

if platform.system() != "Windows":
    raise SystemExit("This script runs only on Windows.")

# 3rd-party libraries
try:
    import keyboard   # pip install keyboard
except Exception:
    raise SystemExit("Install required package: pip install keyboard")

try:
    from pycaw.pycaw import IAudioEndpointVolume, IMMDeviceEnumerator
except Exception:
    raise SystemExit("Install required packages: pip install pycaw comtypes")

# Constants 
#for Microphones
eCapture = 1
eConsole = 0
STEP_PERCENT = 5.0

# Stop event for clean shutdown
stop_event = threading.Event()

# Helper: create IMMDeviceEnumerator (typed)
def _create_mmdevice_enumerator():
    try:
        return CreateObject("MMDeviceEnumerator.MMDeviceEnumerator", interface=IMMDeviceEnumerator)
    except Exception:
        clsid = GUID("{BCDE0395-E52F-467C-8E3D-C4579291692E}")
        return CreateObject(clsid, interface=IMMDeviceEnumerator)

def _get_volume_interface_for_default():
    """
    Returns an IAudioEndpointVolume pointer for the default capture device.
    Caller must ensure COM is initialized in the calling thread.
    """
    enumerator = _create_mmdevice_enumerator()
    default_device = enumerator.GetDefaultAudioEndpoint(eCapture, eConsole)
    iface = default_device.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    return cast(iface, POINTER(IAudioEndpointVolume))

def _percent_to_scalar(p):
    return max(0.0, min(1.0, p / 100.0))

def _scalar_to_percent(s):
    return max(0.0, min(100.0, s * 100.0))

# Decorator to ensure COM initialized per-thread for hotkey handlers
def ensure_com(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        CoInitialize()
        try:
            return func(*args, **kwargs)
        except Exception:
            # Print traceback for easier debugging
            import traceback
            print("Exception in handler:", file=sys.stderr)
            traceback.print_exc()
        finally:
            # Uninitialize COM in this thread when handler completes
            try:
                CoUninitialize()
            except Exception:
                pass
    return wrapper

# Handlers (each will run with COM initialized)
@ensure_com
def increase_volume():
    vol = _get_volume_interface_for_default()
    cur = float(vol.GetMasterVolumeLevelScalar())
    cur_pct = _scalar_to_percent(cur)
    new_pct = min(100.0, cur_pct + STEP_PERCENT)
    vol.SetMasterVolumeLevelScalar(_percent_to_scalar(new_pct), None)
    print(f"[+] Mic volume -> {new_pct:.0f}%")

@ensure_com
def decrease_volume():
    vol = _get_volume_interface_for_default()
    cur = float(vol.GetMasterVolumeLevelScalar())
    cur_pct = _scalar_to_percent(cur)
    new_pct = max(0.0, cur_pct - STEP_PERCENT)
    vol.SetMasterVolumeLevelScalar(_percent_to_scalar(new_pct), None)
    print(f"[-] Mic volume -> {new_pct:.0f}%")

@ensure_com
def toggle_mute():
    vol = _get_volume_interface_for_default()
    cur_mute = bool(vol.GetMute())
    new_mute = not cur_mute
    vol.SetMute(1 if new_mute else 0, None)
    # Print the new state (M when muted, U when unmuted)
    print(f"[{'M' if new_mute else 'U'}] Mic muted -> {new_mute}")

def quit_program():
    """
    Signal the main loop to stop and unhook hotkeys.
    Avoids calling sys.exit() directly from a hotkey handler.
    """
    print("Stopping hotkeys (quit requested)...")
    try:
        keyboard.unhook_all_hotkeys()
    except Exception:
        pass
    stop_event.set()

def register_hotkeys():
    # Avoid duplicate registrations if re-run
    try:
        keyboard.unhook_all_hotkeys()
    except Exception:
        pass

    keyboard.add_hotkey('ctrl+alt+up', increase_volume)
    keyboard.add_hotkey('ctrl+alt+down', decrease_volume)
    keyboard.add_hotkey('ctrl+alt+m', toggle_mute)
    keyboard.add_hotkey('ctrl+alt+q', quit_program)

if __name__ == "__main__":
    print("Registering hotkeys...")
    register_hotkeys()

    # Try to print initial state (initialize COM briefly here)
    CoInitialize()
    try:
        try:
            vol = _get_volume_interface_for_default()
            print(
                f"Initial mic volume: {_scalar_to_percent(float(vol.GetMasterVolumeLevelScalar())):.0f}%"
                f"  muted={bool(vol.GetMute())}"
            )
        except Exception as e:
            print("Could not read initial mic state:", e)
    finally:
        CoUninitialize()

    print("Hotkeys active. Press Ctrl+Alt+Up/Down to change mic, Ctrl+Alt+M to toggle mute, Ctrl+Alt+Q to quit.")
    try:
        # Main loop: wait until quit_program() sets stop_event
        while not stop_event.wait(timeout=0.1):
            # keep the main thread responsive; checks stop_event every 0.1s
            pass
    except KeyboardInterrupt:
        print("Interrupted by user; exiting.")
    finally:
        # clean up in  case
        try:
            keyboard.unhook_all_hotkeys()
        except Exception:
            pass
        print("Exited mic hotkeys.")


Registering hotkeys...
Initial mic volume: 100%  muted=False
Hotkeys active. Press Ctrl+Alt+Up/Down to change mic, Ctrl+Alt+M to toggle mute, Ctrl+Alt+Q to quit.
Exited mic hotkeys.


In [ ]:
"""
speaker_hotkeys_cominit_fixed.py
Global hotkeys to control speaker volume (Windows).
This version ensures COM is initialized inside each hotkey handler thread.

Hotkeys:
  Ctrl+Alt+Up    -> increase speaker volume by 5%
  Ctrl+Alt+Down  -> decrease speaker volume by 5%
  Ctrl+Alt+M     -> toggle mute/unmute
  Ctrl+Alt+Q     -> stop hotkeys (clean shutdown)
"""
import sys
import time
import platform
import threading
from ctypes import POINTER, cast
from comtypes import CLSCTX_ALL, CoInitialize, CoUninitialize
from comtypes.client import CreateObject
from comtypes import GUID
from functools import wraps

if platform.system() != "Windows":
    raise SystemExit("This script runs only on Windows.")

# 3rd-party libraries
try:
    import keyboard   # pip install keyboard
except Exception:
    raise SystemExit("Install required package: pip install keyboard")

try:
    from pycaw.pycaw import IAudioEndpointVolume, IMMDeviceEnumerator
except Exception:
    raise SystemExit("Install required packages: pip install pycaw comtypes")

# Constants
eRender = 0  # Changed from eCapture (1) to eRender (0) for speakers
eConsole = 0
STEP_PERCENT = 5.0

# Stop event for clean shutdown
stop_event = threading.Event()

# Helper: create IMMDeviceEnumerator (typed)
def _create_mmdevice_enumerator():
    try:
        return CreateObject("MMDeviceEnumerator.MMDeviceEnumerator", interface=IMMDeviceEnumerator)
    except Exception:
        clsid = GUID("{BCDE0395-E52F-467C-8E3D-C4579291692E}")
        return CreateObject(clsid, interface=IMMDeviceEnumerator)

def _get_volume_interface_for_default():
    """
    Returns an IAudioEndpointVolume pointer for the default render device (speakers).
    Caller must ensure COM is initialized in the calling thread.
    """
    enumerator = _create_mmdevice_enumerator()
    default_device = enumerator.GetDefaultAudioEndpoint(eRender, eConsole)  # Changed to eRender
    iface = default_device.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    return cast(iface, POINTER(IAudioEndpointVolume))

def _percent_to_scalar(p):
    return max(0.0, min(1.0, p / 100.0))

def _scalar_to_percent(s):
    return max(0.0, min(100.0, s * 100.0))

# Decorator to ensure COM initialized per-thread for hotkey handlers
def ensure_com(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        CoInitialize()
        try:
            return func(*args, **kwargs)
        except Exception:
            # Print traceback for easier debugging
            import traceback
            print("Exception in handler:", file=sys.stderr)
            traceback.print_exc()
        finally:
            # Uninitialize COM in this thread when handler completesæ
            try:
                CoUninitialize()
            except Exception:
                pass
    return wrapper

# Handlers (each will run with COM initialized)
@ensure_com
def increase_volume():
    vol = _get_volume_interface_for_default()
    cur = float(vol.GetMasterVolumeLevelScalar())
    cur_pct = _scalar_to_percent(cur)
    new_pct = min(100.0, cur_pct + STEP_PERCENT)
    vol.SetMasterVolumeLevelScalar(_percent_to_scalar(new_pct), None)
    print(f"[+] Speaker volume -> {new_pct:.0f}%")

@ensure_com
def decrease_volume():
    vol = _get_volume_interface_for_default()
    cur = float(vol.GetMasterVolumeLevelScalar())
    cur_pct = _scalar_to_percent(cur)
    new_pct = max(0.0, cur_pct - STEP_PERCENT)
    vol.SetMasterVolumeLevelScalar(_percent_to_scalar(new_pct), None)
    print(f"[-] Speaker volume -> {new_pct:.0f}%")

@ensure_com
def toggle_mute():
    vol = _get_volume_interface_for_default()
    cur_mute = bool(vol.GetMute())
    new_mute = not cur_mute
    vol.SetMute(1 if new_mute else 0, None)
    # Print the new state (M when muted, U when unmuted)
    print(f"[{'M' if new_mute else 'U'}] Speaker muted -> {new_mute}")

def quit_program():
    """
    Signal the main loop to stop and unhook hotkeys.
    Avoids calling sys.exit() directly from a hotkey handler.
    """
    print("Stopping hotkeys (quit requested)...")
    try:
        keyboard.unhook_all_hotkeys()
    except Exception:
        pass
    stop_event.set()

def register_hotkeys():
    # Avoid duplicate registrations if re-run
    try:
        keyboard.unhook_all_hotkeys()
    except Exception:
        pass

    keyboard.add_hotkey('ctrl+alt+up', increase_volume)
    keyboard.add_hotkey('ctrl+alt+down', decrease_volume)
    keyboard.add_hotkey('ctrl+alt+m', toggle_mute)
    keyboard.add_hotkey('ctrl+alt+q', quit_program)

if __name__ == "__main__":
    print("Registering hotkeys...")
    register_hotkeys()

    # Try to print initial state (initialize COM briefly here)
    CoInitialize()
    try:
        try:
            vol = _get_volume_interface_for_default()
            print(
                f"Initial speaker volume: {_scalar_to_percent(float(vol.GetMasterVolumeLevelScalar())):.0f}%"
                f"  muted={bool(vol.GetMute())}"
            )
        except Exception as e:
            print("Could not read initial speaker state:", e)
    finally:
        CoUninitialize()

    print("Hotkeys active. Press Ctrl+Alt+Up/Down to change volume, Ctrl+Alt+M to toggle mute, Ctrl+Alt+Q to quit.")
    try:
        # Main loop: wait until quit_program() sets stop_event
        while not stop_event.wait(timeout=0.1):
            # keep the main thread responsive; checks stop_event every 0.1s
            pass
    except KeyboardInterrupt:
        print("Interrupted by user; exiting.")
    finally:
        # clean up in any case
        try:
            keyboard.unhook_all_hotkeys()
        except Exception:
            pass
        print("Exited speaker hotkeys.")

In [3]:
"""
speaker_hotkeys_cominit_fixed.py
Global hotkeys to control speaker volume (Windows).
This version ensures COM is initialized inside each hotkey handler thread.

Hotkeys:
  Ctrl+Alt+Up    -> increase speaker volume by 5%
  Ctrl+Alt+Down  -> decrease speaker volume by 5%
  Ctrl+Alt+M     -> toggle mute/unmute
  Ctrl+Alt+Q     -> stop hotkeys (clean shutdown)
"""
import sys
import time
import platform
import threading
from ctypes import POINTER, cast
from comtypes import CLSCTX_ALL, CoInitialize, CoUninitialize
from comtypes.client import CreateObject
from comtypes import GUID
from functools import wraps

if platform.system() != "Windows":
    raise SystemExit("This script runs only on Windows.")

# 3rd-party libraries
try:
    import keyboard   # pip install keyboard
except Exception:
    raise SystemExit("Install required package: pip install keyboard")

try:
    from pycaw.pycaw import IAudioEndpointVolume, IMMDeviceEnumerator
except Exception:
    raise SystemExit("Install required packages: pip install pycaw comtypes")

# Constants
eRender = 0  # Changed from eCapture (1) to eRender (0) for speakers
eConsole = 0
STEP_PERCENT = 5.0

# Stop event for clean shutdown
stop_event = threading.Event()

# Helper: create IMMDeviceEnumerator (typed)
def _create_mmdevice_enumerator():
    try:
        return CreateObject("MMDeviceEnumerator.MMDeviceEnumerator", interface=IMMDeviceEnumerator)
    except Exception:
        clsid = GUID("{BCDE0395-E52F-467C-8E3D-C4579291692E}")
        return CreateObject(clsid, interface=IMMDeviceEnumerator)

def _get_volume_interface_for_default():
    """
    Returns an IAudioEndpointVolume pointer for the default render device (speakers).
    Caller must ensure COM is initialized in the calling thread.
    """
    enumerator = _create_mmdevice_enumerator()
    default_device = enumerator.GetDefaultAudioEndpoint(eRender, eConsole)  # Changed to eRender
    iface = default_device.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    return cast(iface, POINTER(IAudioEndpointVolume))

def _percent_to_scalar(p):
    return max(0.0, min(1.0, p / 100.0))

def _scalar_to_percent(s):
    return max(0.0, min(100.0, s * 100.0))

# Decorator to ensure COM initialized per-thread for hotkey handlers
def ensure_com(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        CoInitialize()
        try:
            return func(*args, **kwargs)
        except Exception:
            # Print traceback for easier debugging
            import traceback
            print("Exception in handler:", file=sys.stderr)
            traceback.print_exc()
        finally:
            # Uninitialize COM in this thread when handler completes
            try:
                CoUninitialize()
            except Exception:
                pass
    return wrapper

# Handlers (each will run with COM initialized)
@ensure_com
def increase_volume():
    vol = _get_volume_interface_for_default()
    cur = float(vol.GetMasterVolumeLevelScalar())
    cur_pct = _scalar_to_percent(cur)
    new_pct = min(100.0, cur_pct + STEP_PERCENT)
    vol.SetMasterVolumeLevelScalar(_percent_to_scalar(new_pct), None)
    print(f"[+] Speaker volume -> {new_pct:.0f}%")

@ensure_com
def decrease_volume():
    vol = _get_volume_interface_for_default()
    cur = float(vol.GetMasterVolumeLevelScalar())
    cur_pct = _scalar_to_percent(cur)
    new_pct = max(0.0, cur_pct - STEP_PERCENT)
    vol.SetMasterVolumeLevelScalar(_percent_to_scalar(new_pct), None)
    print(f"[-] Speaker volume -> {new_pct:.0f}%")

@ensure_com
def toggle_mute():
    vol = _get_volume_interface_for_default()
    cur_mute = bool(vol.GetMute())
    new_mute = not cur_mute
    vol.SetMute(1 if new_mute else 0, None)
    # Print the new state (M when muted, U when unmuted)
    print(f"[{'M' if new_mute else 'U'}] Speaker muted -> {new_mute}")

def quit_program():
    """
    Signal the main loop to stop and unhook hotkeys.
    Avoids calling sys.exit() directly from a hotkey handler.
    """
    print("Stopping hotkeys (quit requested)...")
    try:
        keyboard.unhook_all_hotkeys()
    except Exception:
        pass
    stop_event.set()

def register_hotkeys():
    # Avoid duplicate registrations if re-run
    try:
        keyboard.unhook_all_hotkeys()
    except Exception:
        pass

    keyboard.add_hotkey('ctrl+alt+up', increase_volume)
    keyboard.add_hotkey('ctrl+alt+down', decrease_volume)
    keyboard.add_hotkey('ctrl+alt+m', toggle_mute)
    keyboard.add_hotkey('ctrl+alt+q', quit_program)

if __name__ == "__main__":
    print("=" * 60)
    print("🎵 SPEAKER VOLUME CONTROL - STARTING UP")
    print("=" * 60)
    print("Registering hotkeys...")
    register_hotkeys()

    # Try to print initial state (initialize COM briefly here)
    CoInitialize()
    try:
        try:
            vol = _get_volume_interface_for_default()
            print(
                f"Initial speaker volume: {_scalar_to_percent(float(vol.GetMasterVolumeLevelScalar())):.0f}%"
                f"  muted={bool(vol.GetMute())}"
            )
        except Exception as e:
            print("Could not read initial speaker state:", e)
    finally:
        CoUninitialize()

    print("\n✅ HOTKEYS ARE NOW ACTIVE!")
    print("=" * 60)
    print("  Ctrl+Alt+Up    -> Increase volume")
    print("  Ctrl+Alt+Down  -> Decrease volume")
    print("  Ctrl+Alt+M     -> Toggle mute")
    print("  Ctrl+Alt+Q     -> Quit program")
    print("=" * 60)
    print("⏳ Running... (Press Ctrl+Alt+Q to stop)")
    print("=" * 60)
    
    # Heartbeat counter to show it's alive
    heartbeat_counter = 0
    try:
        # Main loop: wait until quit_program() sets stop_event
        while not stop_event.wait(timeout=5.0):  # Changed to 5s for heartbeat
            heartbeat_counter += 1
            # Print a heartbeat every 5 seconds to show it's running
            print(f"* Alive [{heartbeat_counter * 5}s] - Listening for hotkeys...")
    except KeyboardInterrupt:
        print("\n⚠️  Interrupted by user; exiting.")
    finally:
        # clean up in any case
        try:
            keyboard.unhook_all_hotkeys()
        except Exception:
            pass
        print("\n" + "=" * 60)
        print("🛑 SPEAKER HOTKEYS STOPPED")
        print("=" * 60)

🎵 SPEAKER VOLUME CONTROL - STARTING UP
Registering hotkeys...
Initial speaker volume: 100%  muted=False

✅ HOTKEYS ARE NOW ACTIVE!
  Ctrl+Alt+Up    -> Increase volume
  Ctrl+Alt+Down  -> Decrease volume
  Ctrl+Alt+M     -> Toggle mute
  Ctrl+Alt+Q     -> Quit program
⏳ Running... (Press Ctrl+Alt+Q to stop)
* Alive [5s] - Listening for hotkeys...
* Alive [10s] - Listening for hotkeys...
* Alive [15s] - Listening for hotkeys...
* Alive [20s] - Listening for hotkeys...
* Alive [25s] - Listening for hotkeys...
* Alive [30s] - Listening for hotkeys...

🛑 SPEAKER HOTKEYS STOPPED


In [1]:
# for speakers---->
"""
speaker_volume_gui_jupyter.py
GUI interface to control speaker volume - Jupyter Notebook compatible (Windows).

Run in Jupyter:
    %run speaker_volume_gui_jupyter.py

Control from other cells:
    stop_gui()     # Stop the GUI
    is_gui_running()  # Check if running
"""
import sys
import platform
import threading
from ctypes import POINTER, cast
from comtypes import CLSCTX_ALL, CoInitialize, CoUninitialize
from comtypes.client import CreateObject
from comtypes import GUID
from functools import wraps

if platform.system() != "Windows":
    raise SystemExit("This script runs only on Windows.")

try:
    import keyboard
    import tkinter as tk
    from tkinter import ttk
except Exception as e:
    raise SystemExit(f"Install required packages: pip install keyboard\nError: {e}")

try:
    from pycaw.pycaw import IAudioEndpointVolume, IMMDeviceEnumerator
except Exception:
    raise SystemExit("Install required packages: pip install pycaw comtypes")

# Constants
eRender = 0
eConsole = 0
STEP_PERCENT = 5.0

# Global reference to GUI instance
_gui_instance = None
_gui_thread = None
_is_running = False

class SpeakerVolumeGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("🔊 Speaker Volume Control")
        self.root.geometry("450x500")
        self.root.resizable(False, False)
        
        self.hotkeys_active = False
        self.update_job = None
        self.should_close = False
        
        # Initialize COM for this thread
        CoInitialize()
        
        # Setup UI
        self.setup_ui()
        
        # Start hotkeys
        self.start_hotkeys()
        
        # Start updating display
        self.update_display()
        
        # Handle window close
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)
    
    def setup_ui(self):
        # Header
        header_frame = tk.Frame(self.root, bg="#2c3e50", height=60)
        header_frame.pack(fill=tk.X)
        header_frame.pack_propagate(False)
        
        title_label = tk.Label(
            header_frame,
            text="🔊 Speaker Volume Control",
            font=("Arial", 16, "bold"),
            bg="#2c3e50",
            fg="white"
        )
        title_label.pack(pady=15)
        
        # Main content frame
        content_frame = tk.Frame(self.root, bg="white")
        content_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)
        
        # Volume display
        volume_frame = tk.LabelFrame(
            content_frame,
            text="Current Volume",
            font=("Arial", 10, "bold"),
            bg="white",
            fg="#2c3e50"
        )
        volume_frame.pack(fill=tk.X, pady=(0, 15))
        
        self.volume_label = tk.Label(
            volume_frame,
            text="---%",
            font=("Arial", 36, "bold"),
            bg="white",
            fg="#27ae60"
        )
        self.volume_label.pack(pady=15)
        
        # Progress bar
        self.volume_progress = ttk.Progressbar(
            volume_frame,
            length=350,
            mode='determinate',
            maximum=100
        )
        self.volume_progress.pack(pady=(0, 15))
        
        # Mute status
        self.mute_label = tk.Label(
            volume_frame,
            text="🔊 Unmuted",
            font=("Arial", 12),
            bg="white",
            fg="#27ae60"
        )
        self.mute_label.pack(pady=(0, 10))
        
        # Manual controls
        control_frame = tk.LabelFrame(
            content_frame,
            text="Manual Controls",
            font=("Arial", 10, "bold"),
            bg="white",
            fg="#2c3e50"
        )
        control_frame.pack(fill=tk.X, pady=(0, 15))
        
        buttons_frame = tk.Frame(control_frame, bg="white")
        buttons_frame.pack(pady=15)
        
        # Volume buttons
        self.btn_vol_down = tk.Button(
            buttons_frame,
            text="🔉 -5%",
            command=self.decrease_volume,
            font=("Arial", 11),
            bg="#e74c3c",
            fg="white",
            width=10,
            cursor="hand2"
        )
        self.btn_vol_down.pack(side=tk.LEFT, padx=5)
        
        self.btn_mute = tk.Button(
            buttons_frame,
            text="🔇 Mute",
            command=self.toggle_mute,
            font=("Arial", 11),
            bg="#95a5a6",
            fg="white",
            width=10,
            cursor="hand2"
        )
        self.btn_mute.pack(side=tk.LEFT, padx=5)
        
        self.btn_vol_up = tk.Button(
            buttons_frame,
            text="🔊 +5%",
            command=self.increase_volume,
            font=("Arial", 11),
            bg="#27ae60",
            fg="white",
            width=10,
            cursor="hand2"
        )
        self.btn_vol_up.pack(side=tk.LEFT, padx=5)
        
        # Hotkeys info
        hotkey_frame = tk.LabelFrame(
            content_frame,
            text="Hotkeys",
            font=("Arial", 10, "bold"),
            bg="white",
            fg="#2c3e50"
        )
        hotkey_frame.pack(fill=tk.X, pady=(0, 15))
        
        hotkeys_info = [
            ("Ctrl+Alt+Up", "Increase Volume"),
            ("Ctrl+Alt+Down", "Decrease Volume"),
            ("Ctrl+Alt+M", "Toggle Mute"),
        ]
        
        for hotkey, action in hotkeys_info:
            hk_frame = tk.Frame(hotkey_frame, bg="white")
            hk_frame.pack(fill=tk.X, padx=15, pady=5)
            
            tk.Label(
                hk_frame,
                text=hotkey,
                font=("Courier", 10, "bold"),
                bg="white",
                fg="#3498db",
                width=20,
                anchor="w"
            ).pack(side=tk.LEFT)
            
            tk.Label(
                hk_frame,
                text=action,
                font=("Arial", 10),
                bg="white",
                fg="#7f8c8d",
                anchor="w"
            ).pack(side=tk.LEFT)
        
        # Status bar
        status_frame = tk.Frame(content_frame, bg="white")
        status_frame.pack(fill=tk.X)
        
        self.status_label = tk.Label(
            status_frame,
            text="✅ Hotkeys Active | Running in Jupyter",
            font=("Arial", 9),
            bg="#d4edda",
            fg="#155724",
            relief=tk.FLAT,
            padx=10,
            pady=8
        )
        self.status_label.pack(fill=tk.X)
        
        # Jupyter tip
        jupyter_tip = tk.Label(
            status_frame,
            text="💡 Tip: Use stop_gui() in another cell to close",
            font=("Arial", 8, "italic"),
            bg="white",
            fg="#6c757d",
            pady=5
        )
        jupyter_tip.pack()
    
    def _create_mmdevice_enumerator(self):
        try:
            return CreateObject("MMDeviceEnumerator.MMDeviceEnumerator", interface=IMMDeviceEnumerator)
        except Exception:
            clsid = GUID("{BCDE0395-E52F-467C-8E3D-C4579291692E}")
            return CreateObject(clsid, interface=IMMDeviceEnumerator)
    
    def _get_volume_interface(self):
        enumerator = self._create_mmdevice_enumerator()
        default_device = enumerator.GetDefaultAudioEndpoint(eRender, eConsole)
        iface = default_device.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
        return cast(iface, POINTER(IAudioEndpointVolume))
    
    def _percent_to_scalar(self, p):
        return max(0.0, min(1.0, p / 100.0))
    
    def _scalar_to_percent(self, s):
        return max(0.0, min(100.0, s * 100.0))
    
    def get_current_volume(self):
        try:
            vol = self._get_volume_interface()
            scalar = float(vol.GetMasterVolumeLevelScalar())
            return self._scalar_to_percent(scalar)
        except Exception as e:
            print(f"Error getting volume: {e}")
            return 0
    
    def get_mute_status(self):
        try:
            vol = self._get_volume_interface()
            return bool(vol.GetMute())
        except Exception as e:
            print(f"Error getting mute status: {e}")
            return False
    
    def increase_volume(self):
        try:
            vol = self._get_volume_interface()
            cur = float(vol.GetMasterVolumeLevelScalar())
            cur_pct = self._scalar_to_percent(cur)
            new_pct = min(100.0, cur_pct + STEP_PERCENT)
            vol.SetMasterVolumeLevelScalar(self._percent_to_scalar(new_pct), None)
        except Exception as e:
            print(f"Error increasing volume: {e}")
    
    def decrease_volume(self):
        try:
            vol = self._get_volume_interface()
            cur = float(vol.GetMasterVolumeLevelScalar())
            cur_pct = self._scalar_to_percent(cur)
            new_pct = max(0.0, cur_pct - STEP_PERCENT)
            vol.SetMasterVolumeLevelScalar(self._percent_to_scalar(new_pct), None)
        except Exception as e:
            print(f"Error decreasing volume: {e}")
    
    def toggle_mute(self):
        try:
            vol = self._get_volume_interface()
            cur_mute = bool(vol.GetMute())
            new_mute = not cur_mute
            vol.SetMute(1 if new_mute else 0, None)
        except Exception as e:
            print(f"Error toggling mute: {e}")
    
    def update_display(self):
        if self.should_close:
            return
            
        try:
            volume = self.get_current_volume()
            is_muted = self.get_mute_status()
            
            # Update volume label and progress
            self.volume_label.config(text=f"{volume:.0f}%")
            self.volume_progress['value'] = volume
            
            # Update mute status
            if is_muted:
                self.mute_label.config(text="🔇 Muted", fg="#e74c3c")
                self.volume_label.config(fg="#e74c3c")
                self.btn_mute.config(text="🔊 Unmute", bg="#27ae60")
            else:
                self.mute_label.config(text="🔊 Unmuted", fg="#27ae60")
                self.volume_label.config(fg="#27ae60")
                self.btn_mute.config(text="🔇 Mute", bg="#95a5a6")
        except Exception as e:
            print(f"Error updating display: {e}")
        
        # Schedule next update
        self.update_job = self.root.after(500, self.update_display)
    
    def ensure_com(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            CoInitialize()
            try:
                return func(*args, **kwargs)
            except Exception as e:
                print(f"Error in hotkey handler: {e}")
            finally:
                try:
                    CoUninitialize()
                except Exception:
                    pass
        return wrapper
    
    def start_hotkeys(self):
        try:
            keyboard.unhook_all_hotkeys()
        except Exception:
            pass
        
        # Register hotkeys with COM initialization
        keyboard.add_hotkey('ctrl+alt+up', self.ensure_com(self.increase_volume))
        keyboard.add_hotkey('ctrl+alt+down', self.ensure_com(self.decrease_volume))
        keyboard.add_hotkey('ctrl+alt+m', self.ensure_com(self.toggle_mute))
        
        self.hotkeys_active = True
    
    def stop_hotkeys(self):
        try:
            keyboard.unhook_all_hotkeys()
            self.hotkeys_active = False
        except Exception as e:
            print(f"Error stopping hotkeys: {e}")
    
    def on_closing(self):
        global _is_running, _gui_instance
        self.should_close = True
        if self.update_job:
            self.root.after_cancel(self.update_job)
        self.stop_hotkeys()
        try:
            CoUninitialize()
        except Exception:
            pass
        _is_running = False
        _gui_instance = None
        self.root.destroy()
        print("🛑 Speaker Volume GUI closed")

def run_gui():
    """Run the GUI in a separate thread"""
    global _gui_instance, _is_running
    
    root = tk.Tk()
    _gui_instance = SpeakerVolumeGUI(root)
    _is_running = True
    
    print("✅ Speaker Volume GUI started!")
    print("💡 Use stop_gui() in another cell to close the window")
    
    root.mainloop()
    
    _is_running = False
    _gui_instance = None

def start_gui():
    """Start the GUI in a background thread (for Jupyter)"""
    global _gui_thread, _is_running
    
    if _is_running:
        print("⚠️  GUI is already running!")
        return
    
    _gui_thread = threading.Thread(target=run_gui, daemon=True)
    _gui_thread.start()

def stop_gui():
    """Stop the GUI from another Jupyter cell"""
    global _gui_instance, _is_running
    
    if not _is_running or _gui_instance is None:
        print("⚠️  GUI is not running")
        return
    
    try:
        _gui_instance.root.quit()
        print("🛑 Closing GUI...")
    except Exception as e:
        print(f"Error stopping GUI: {e}")

def is_gui_running():
    """Check if GUI is currently running"""
    return _is_running

# Auto-start when script is run
if __name__ == "__main__":
    print("=" * 60)
    print("🔊 SPEAKER VOLUME CONTROL - JUPYTER MODE")
    print("=" * 60)
    start_gui()
    print("\n📋 Available commands in other cells:")
    print("   • stop_gui()        - Close the GUI window")
    print("   • is_gui_running()  - Check if GUI is running")
    print("=" * 60)

🔊 SPEAKER VOLUME CONTROL - JUPYTER MODE

📋 Available commands in other cells:
   • stop_gui()        - Close the GUI window
   • is_gui_running()  - Check if GUI is running
✅ Speaker Volume GUI started!
💡 Use stop_gui() in another cell to close the window
🛑 Speaker Volume GUI closed


In [3]:
# for Microphones----->
"""
mic_volume_gui_jupyter.py
GUI interface to control microphone volume - Jupyter Notebook compatible (Windows).

Run in Jupyter:
    %run mic_volume_gui_jupyter.py

Control from other cells:
    stop_gui()     # Stop the GUI
    is_gui_running()  # Check if running
"""
import sys
import platform
import threading
from ctypes import POINTER, cast
from comtypes import CLSCTX_ALL, CoInitialize, CoUninitialize
from comtypes.client import CreateObject
from comtypes import GUID
from functools import wraps

if platform.system() != "Windows":
    raise SystemExit("This script runs only on Windows.")

try:
    import keyboard
    import tkinter as tk
    from tkinter import ttk
except Exception as e:
    raise SystemExit(f"Install required packages: pip install keyboard\nError: {e}")

try:
    from pycaw.pycaw import IAudioEndpointVolume, IMMDeviceEnumerator
except Exception:
    raise SystemExit("Install required packages: pip install pycaw comtypes")

# Constants
eCapture = 1  # Microphone (capture device)
eConsole = 0
STEP_PERCENT = 5.0

# Global reference to GUI instance
_gui_instance = None
_gui_thread = None
_is_running = False

class MicVolumeGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("🎤 Microphone Volume Control")
        self.root.geometry("450x500")
        self.root.resizable(False, False)
        
        self.hotkeys_active = False
        self.update_job = None
        self.should_close = False
        
        # Initialize COM for this thread
        CoInitialize()
        
        # Setup UI
        self.setup_ui()
        
        # Start hotkeys
        self.start_hotkeys()
        
        # Start updating display
        self.update_display()
        
        # Handle window close
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)
    
    def setup_ui(self):
        # Header
        header_frame = tk.Frame(self.root, bg="#8e44ad", height=60)
        header_frame.pack(fill=tk.X)
        header_frame.pack_propagate(False)
        
        title_label = tk.Label(
            header_frame,
            text="🎤 Microphone Volume Control",
            font=("Arial", 16, "bold"),
            bg="#8e44ad",
            fg="white"
        )
        title_label.pack(pady=15)
        
        # Main content frame
        content_frame = tk.Frame(self.root, bg="white")
        content_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)
        
        # Volume display
        volume_frame = tk.LabelFrame(
            content_frame,
            text="Current Microphone Volume",
            font=("Arial", 10, "bold"),
            bg="white",
            fg="#8e44ad"
        )
        volume_frame.pack(fill=tk.X, pady=(0, 15))
        
        self.volume_label = tk.Label(
            volume_frame,
            text="---%",
            font=("Arial", 36, "bold"),
            bg="white",
            fg="#9b59b6"
        )
        self.volume_label.pack(pady=15)
        
        # Progress bar
        style = ttk.Style()
        style.configure("Mic.Horizontal.TProgressbar", background='#9b59b6')
        
        self.volume_progress = ttk.Progressbar(
            volume_frame,
            length=350,
            mode='determinate',
            maximum=100,
            style="Mic.Horizontal.TProgressbar"
        )
        self.volume_progress.pack(pady=(0, 15))
        
        # Mute status
        self.mute_label = tk.Label(
            volume_frame,
            text="🎤 Unmuted",
            font=("Arial", 12),
            bg="white",
            fg="#27ae60"
        )
        self.mute_label.pack(pady=(0, 10))
        
        # Manual controls
        control_frame = tk.LabelFrame(
            content_frame,
            text="Manual Controls",
            font=("Arial", 10, "bold"),
            bg="white",
            fg="#8e44ad"
        )
        control_frame.pack(fill=tk.X, pady=(0, 15))
        
        buttons_frame = tk.Frame(control_frame, bg="white")
        buttons_frame.pack(pady=15)
        
        # Volume buttons
        self.btn_vol_down = tk.Button(
            buttons_frame,
            text="🔉 -5%",
            command=self.decrease_volume,
            font=("Arial", 11),
            bg="#e74c3c",
            fg="white",
            width=10,
            cursor="hand2"
        )
        self.btn_vol_down.pack(side=tk.LEFT, padx=5)
        
        self.btn_mute = tk.Button(
            buttons_frame,
            text="🔇 Mute",
            command=self.toggle_mute,
            font=("Arial", 11),
            bg="#95a5a6",
            fg="white",
            width=10,
            cursor="hand2"
        )
        self.btn_mute.pack(side=tk.LEFT, padx=5)
        
        self.btn_vol_up = tk.Button(
            buttons_frame,
            text="🔊 +5%",
            command=self.increase_volume,
            font=("Arial", 11),
            bg="#9b59b6",
            fg="white",
            width=10,
            cursor="hand2"
        )
        self.btn_vol_up.pack(side=tk.LEFT, padx=5)
        
        # Hotkeys info
        hotkey_frame = tk.LabelFrame(
            content_frame,
            text="Hotkeys",
            font=("Arial", 10, "bold"),
            bg="white",
            fg="#8e44ad"
        )
        hotkey_frame.pack(fill=tk.X, pady=(0, 15))
        
        hotkeys_info = [
            ("Ctrl+Alt+Up", "Increase Mic Volume"),
            ("Ctrl+Alt+Down", "Decrease Mic Volume"),
            ("Ctrl+Alt+M", "Toggle Mute"),
        ]
        
        for hotkey, action in hotkeys_info:
            hk_frame = tk.Frame(hotkey_frame, bg="white")
            hk_frame.pack(fill=tk.X, padx=15, pady=5)
            
            tk.Label(
                hk_frame,
                text=hotkey,
                font=("Courier", 10, "bold"),
                bg="white",
                fg="#9b59b6",
                width=20,
                anchor="w"
            ).pack(side=tk.LEFT)
            
            tk.Label(
                hk_frame,
                text=action,
                font=("Arial", 10),
                bg="white",
                fg="#7f8c8d",
                anchor="w"
            ).pack(side=tk.LEFT)
        
        # Status bar
        status_frame = tk.Frame(content_frame, bg="white")
        status_frame.pack(fill=tk.X)
        
        self.status_label = tk.Label(
            status_frame,
            text="✅ Hotkeys Active | Running in Jupyter",
            font=("Arial", 9),
            bg="#e8daef",
            fg="#6c3483",
            relief=tk.FLAT,
            padx=10,
            pady=8
        )
        self.status_label.pack(fill=tk.X)
        
        # Jupyter tip
        jupyter_tip = tk.Label(
            status_frame,
            text="💡 Tip: Use stop_gui() in another cell to close",
            font=("Arial", 8, "italic"),
            bg="white",
            fg="#6c757d",
            pady=5
        )
        jupyter_tip.pack()
    
    def _create_mmdevice_enumerator(self):
        try:
            return CreateObject("MMDeviceEnumerator.MMDeviceEnumerator", interface=IMMDeviceEnumerator)
        except Exception:
            clsid = GUID("{BCDE0395-E52F-467C-8E3D-C4579291692E}")
            return CreateObject(clsid, interface=IMMDeviceEnumerator)
    
    def _get_volume_interface(self):
        enumerator = self._create_mmdevice_enumerator()
        default_device = enumerator.GetDefaultAudioEndpoint(eCapture, eConsole)  # Microphone
        iface = default_device.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
        return cast(iface, POINTER(IAudioEndpointVolume))
    
    def _percent_to_scalar(self, p):
        return max(0.0, min(1.0, p / 100.0))
    
    def _scalar_to_percent(self, s):
        return max(0.0, min(100.0, s * 100.0))
    
    def get_current_volume(self):
        try:
            vol = self._get_volume_interface()
            scalar = float(vol.GetMasterVolumeLevelScalar())
            return self._scalar_to_percent(scalar)
        except Exception as e:
            print(f"Error getting mic volume: {e}")
            return 0
    
    def get_mute_status(self):
        try:
            vol = self._get_volume_interface()
            return bool(vol.GetMute())
        except Exception as e:
            print(f"Error getting mute status: {e}")
            return False
    
    def increase_volume(self):
        try:
            vol = self._get_volume_interface()
            cur = float(vol.GetMasterVolumeLevelScalar())
            cur_pct = self._scalar_to_percent(cur)
            new_pct = min(100.0, cur_pct + STEP_PERCENT)
            vol.SetMasterVolumeLevelScalar(self._percent_to_scalar(new_pct), None)
        except Exception as e:
            print(f"Error increasing mic volume: {e}")
    
    def decrease_volume(self):
        try:
            vol = self._get_volume_interface()
            cur = float(vol.GetMasterVolumeLevelScalar())
            cur_pct = self._scalar_to_percent(cur)
            new_pct = max(0.0, cur_pct - STEP_PERCENT)
            vol.SetMasterVolumeLevelScalar(self._percent_to_scalar(new_pct), None)
        except Exception as e:
            print(f"Error decreasing mic volume: {e}")
    
    def toggle_mute(self):
        try:
            vol = self._get_volume_interface()
            cur_mute = bool(vol.GetMute())
            new_mute = not cur_mute
            vol.SetMute(1 if new_mute else 0, None)
        except Exception as e:
            print(f"Error toggling mic mute: {e}")
    
    def update_display(self):
        if self.should_close:
            return
            
        try:
            volume = self.get_current_volume()
            is_muted = self.get_mute_status()
            
            # Update volume label and progress
            self.volume_label.config(text=f"{volume:.0f}%")
            self.volume_progress['value'] = volume
            
            # Update mute status
            if is_muted:
                self.mute_label.config(text="🔇 Muted", fg="#e74c3c")
                self.volume_label.config(fg="#e74c3c")
                self.btn_mute.config(text="🎤 Unmute", bg="#27ae60")
            else:
                self.mute_label.config(text="🎤 Unmuted", fg="#27ae60")
                self.volume_label.config(fg="#9b59b6")
                self.btn_mute.config(text="🔇 Mute", bg="#95a5a6")
        except Exception as e:
            print(f"Error updating display: {e}")
        
        # Schedule next update
        self.update_job = self.root.after(500, self.update_display)
    
    def ensure_com(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            CoInitialize()
            try:
                return func(*args, **kwargs)
            except Exception as e:
                print(f"Error in hotkey handler: {e}")
            finally:
                try:
                    CoUninitialize()
                except Exception:
                    pass
        return wrapper
    
    def start_hotkeys(self):
        try:
            keyboard.unhook_all_hotkeys()
        except Exception:
            pass
        
        # Register hotkeys with COM initialization
        keyboard.add_hotkey('ctrl+alt+up', self.ensure_com(self.increase_volume))
        keyboard.add_hotkey('ctrl+alt+down', self.ensure_com(self.decrease_volume))
        keyboard.add_hotkey('ctrl+alt+m', self.ensure_com(self.toggle_mute))
        
        self.hotkeys_active = True
    
    def stop_hotkeys(self):
        try:
            keyboard.unhook_all_hotkeys()
            self.hotkeys_active = False
        except Exception as e:
            print(f"Error stopping hotkeys: {e}")
    
    def on_closing(self):
        global _is_running, _gui_instance
        self.should_close = True
        if self.update_job:
            self.root.after_cancel(self.update_job)
        self.stop_hotkeys()
        try:
            CoUninitialize()
        except Exception:
            pass
        _is_running = False
        _gui_instance = None
        self.root.destroy()
        print("🛑 Microphone Volume GUI closed")

def run_gui():
    """Run the GUI in a separate thread"""
    global _gui_instance, _is_running
    
    root = tk.Tk()
    _gui_instance = MicVolumeGUI(root)
    _is_running = True
    
    print("✅ Microphone Volume GUI started!")
    print("💡 Use stop_gui() in another cell to close the window")
    
    root.mainloop()
    
    _is_running = False
    _gui_instance = None

def start_gui():
    """Start the GUI in a background thread (for Jupyter)"""
    global _gui_thread, _is_running
    
    if _is_running:
        print("⚠️  GUI is already running!")
        return
    
    _gui_thread = threading.Thread(target=run_gui, daemon=True)
    _gui_thread.start()

def stop_gui():
    """Stop the GUI from another Jupyter cell"""
    global _gui_instance, _is_running
    
    if not _is_running or _gui_instance is None:
        print("⚠️  GUI is not running")
        return
    
    try:
        _gui_instance.root.quit()
        print("🛑 Closing GUI...")
    except Exception as e:
        print(f"Error stopping GUI: {e}")

def is_gui_running():
    """Check if GUI is currently running"""
    return _is_running

# Auto-start when script is run
if __name__ == "__main__":
    print("=" * 60)
    print("🎤 MICROPHONE VOLUME CONTROL - JUPYTER MODE")
    print("=" * 60)
    start_gui()
    print("\n📋 Available commands in other cells:")
    print("   • stop_gui()        - Close the GUI window")
    print("   • is_gui_running()  - Check if GUI is running")
    print("=" * 60)

🎤 MICROPHONE VOLUME CONTROL - JUPYTER MODE

📋 Available commands in other cells:
   • stop_gui()        - Close the GUI window
   • is_gui_running()  - Check if GUI is running
✅ Microphone Volume GUI started!
💡 Use stop_gui() in another cell to close the window
